# Homework 2: training pipeline

This code will test your homework 2 solutions by using them in a complete ML pipeline. You should run this code in order to tune your model and save your model weights (which will also be uploaded as part of your solution)

In [1]:
# Download the training data from the homework2 folder:
# unzip using tar xzvvf nsynth_subset.tar.gz
# (this is a small subset of the "nsynth" dataset: https://magenta.tensorflow.org/datasets/nsynth)

In [1]:
import homework2

821
[0, 0, 1, 1, 0]


### Install and Load Required Libraries  

In [3]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [5]:
BATCH_SIZE = 16
torch.use_deterministic_algorithms(True)

In [6]:
if not len(homework2.audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [7]:
# Some helper functions. These are the same as what the autograder runs.

In [8]:
# Split dataset into train / valid / test
def split_data(waveforms, labels, train_ratio=0.7, valid_ratio=0.15):
    assert(train_ratio + valid_ratio < 1)
    test_ratio = 1 - (train_ratio + valid_ratio)
    N = len(waveforms)
    Ntrain = int(N * train_ratio)
    Nvalid = int(N * valid_ratio)
    Ntest = int(N * test_ratio)
    Wtrain = waveforms[:Ntrain]
    Wvalid = waveforms[Ntrain:Ntrain + Nvalid]
    Wtest = waveforms[Ntrain + Nvalid:]
    ytrain = labels[:Ntrain]
    yvalid = labels[Ntrain:Ntrain + Nvalid]
    ytest = labels[Ntrain + Nvalid:]
    return Wtrain,Wvalid,Wtest,ytrain,yvalid,ytest

In [9]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [10]:
class InstrumentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        return features, torch.tensor(label, dtype=torch.long)

In [11]:
class Loaders():
    def __init__(self, waveforms, labels, feature_function, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        self.Wtrain, self.Wvalid, self.Wtest, self.ytrain, self.yvalid, self.ytest = split_data(waveforms, labels)
        
        self.Xtrain = process_data(self.Wtrain, feature_function)
        self.Xvalid = process_data(self.Wvalid, feature_function)
        self.Xtest = process_data(self.Wtest, feature_function)
        
        self.dataTrain = InstrumentDataset(self.Xtrain, self.ytrain)
        self.dataValid = InstrumentDataset(self.Xvalid, self.yvalid)
        self.dataTest = InstrumentDataset(self.Xtest, self.ytest)
        
        self.loaderTrain = DataLoader(self.dataTrain, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderValid = DataLoader(self.dataValid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(self.dataTest, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [12]:
class Pipeline():
    def __init__(self, module, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.criterion = nn.CrossEntropyLoss()
        
        self.model = module.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def evaluate(self, loader, which = "valid"):
        self.model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                #loss = criterion(outputs, labels) # validation loss

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        
        return acc
    
    def train(self, loaders,
          num_epochs=1, # Train for a single epoch by default
          model_path=None): # (Optionally) provide a path to save the best model
        val_acc = 0
        best_val_acc = 0
        for epoch in range(num_epochs):
            self.model.train()
            
            losses = []

            for inputs, labels in loaders.loaderTrain:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                losses.append(float(loss))
            
            self.model.eval()
            val_acc = self.evaluate(loaders.loaderValid)
            print("Epoch " + str(epoch) + ", loss = " + str(sum(losses)/len(losses)) +\
                  ", validation accuracy = " + str(val_acc))

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                if (model_path):
                    torch.save(self.model.state_dict(), model_path)
        print("Final validation accuracy = " + str(val_acc) + ", best = " + str(best_val_acc))
        return val_acc, best_val_acc

    def load(self, path):
        self.model.load_state_dict(torch.load(path, weights_only=True))

In [13]:
# The function below is the basis of how the autograder tests your code. Try to understand this one.

In [14]:
def test(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 10, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [16]:
# 1. Paths, labels, waveforms

In [17]:
# Once you've written the corresponding code in homework2.py, print these out or visualize them if you want
homework2.waveforms
homework2.labels

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,


In [18]:
# 2. MFCC

In [19]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mfcc,
     homework2.MLPClassifier(),
     0.0001,
     "best_mlp_model.weights")

Extracting features...
Epoch 0, loss = 0.7292926046583388, validation accuracy = 0.8211382113821138
Epoch 1, loss = 0.3977507911622524, validation accuracy = 0.9186991869918699
Epoch 2, loss = 0.2475849977797932, validation accuracy = 0.967479674796748
Epoch 3, loss = 0.16193912861247858, validation accuracy = 0.983739837398374
Epoch 4, loss = 0.11104533852388461, validation accuracy = 0.983739837398374
Epoch 5, loss = 0.08307227657900916, validation accuracy = 0.991869918699187
Epoch 6, loss = 0.06746191784946455, validation accuracy = 0.991869918699187
Epoch 7, loss = 0.057069568843063384, validation accuracy = 0.991869918699187
Epoch 8, loss = 0.04932543537062076, validation accuracy = 0.991869918699187
Epoch 9, loss = 0.04333182928773264, validation accuracy = 0.991869918699187
Final validation accuracy = 0.991869918699187, best = 0.991869918699187
Test accuracy = 0.9838709677419355


In [20]:
# 3. Spectrogram

In [21]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_spec,
     homework2.SimpleCNN(),
     0.0001,
     "best_spec_model.weights")

Extracting features...
Epoch 0, loss = 0.6326414032114877, validation accuracy = 0.9024390243902439
Epoch 1, loss = 0.5699289507336087, validation accuracy = 0.9186991869918699
Epoch 2, loss = 0.5416444970501794, validation accuracy = 0.9186991869918699
Epoch 3, loss = 0.519173677596781, validation accuracy = 0.9186991869918699
Epoch 4, loss = 0.4941016659140587, validation accuracy = 0.9186991869918699
Epoch 5, loss = 0.4656078724397553, validation accuracy = 0.926829268292683
Epoch 6, loss = 0.4362041743265258, validation accuracy = 0.926829268292683
Epoch 7, loss = 0.4084658059808943, validation accuracy = 0.959349593495935
Epoch 8, loss = 0.38322650144497555, validation accuracy = 0.959349593495935
Epoch 9, loss = 0.36003505024645066, validation accuracy = 0.967479674796748
Final validation accuracy = 0.967479674796748, best = 0.967479674796748
Test accuracy = 0.8387096774193549


In [ ]:
# 4. Mel-spectrogram

In [22]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mel,
     homework2.SimpleCNN(),
     0.0001,
     "best_mel_model.weights")

Extracting features...
Epoch 0, loss = 0.4973374845253097, validation accuracy = 0.8211382113821138
Epoch 1, loss = 0.3461121954023838, validation accuracy = 0.8861788617886179
Epoch 2, loss = 0.28981877863407135, validation accuracy = 0.926829268292683
Epoch 3, loss = 0.25354652148154044, validation accuracy = 0.959349593495935
Epoch 4, loss = 0.2260940989686383, validation accuracy = 0.975609756097561
Epoch 5, loss = 0.20433692675497797, validation accuracy = 0.991869918699187
Epoch 6, loss = 0.18600386877854666, validation accuracy = 1.0
Epoch 7, loss = 0.16972039764126143, validation accuracy = 1.0
Epoch 8, loss = 0.15508571474088562, validation accuracy = 1.0
Epoch 9, loss = 0.14196641163693535, validation accuracy = 1.0
Final validation accuracy = 1.0, best = 1.0
Test accuracy = 0.9596774193548387


In [23]:
# 5. Constant-Q

In [15]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_q_model.weights")

Extracting features...
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code


/Users/mehul/Documents/UCSD Classes/ML for Music/MLforMusic/homework2/homework2.py:303: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:308.)
  return torch.FloatTensor(result)


new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
n

In [ ]:
# 6. Pitch shift

In [16]:
test(homework2.augmented_waveforms,
     homework2.augmented_labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_augmented_model.weights")

Extracting features...
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new code
new c

In [ ]:
# 7. Extend your model to handle four classes and creatively improve its performance

In [ ]:
test(homework2.waveforms,
     homework2.labels_7,
     homework2.feature_func_7,
     homework2.model_7,
     0.0001,
     "best_model_7.weights")